# Medical Image Captioning - Model Training

This notebook trains an encoder-decoder model with attention mechanism for generating medical image captions from chest X-rays.

## Architecture Overview

- **Encoder**: DenseNet-121 pretrained on ImageNet, fine-tuned for medical images
- **Decoder**: LSTM with Bahdanau (additive) attention mechanism
- **Vocabulary**: 514 tokens from Indiana University Chest X-Ray dataset

## Training Objectives

1. **Minimize Cross-Entropy Loss**: Learn to predict correct word sequences
2. **Maximize BLEU/METEOR/ROUGE-L**: Generate captions similar to radiologist reports
3. **Learn Meaningful Attention**: Focus on relevant anatomical regions

---

## 1. Setup

In [1]:
# Check execution environment
import sys
from pathlib import Path
import os, yaml, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Add src to path for imports
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Working directory: {Path.cwd()}")

Project root: /Users/marialinhares/uni/masters/1year1semester/MCIM/MCIM_Final_Project
Working directory: /Users/marialinhares/uni/masters/1year1semester/MCIM/MCIM_Final_Project/notebooks


In [2]:
# Import project modules
from src.models.caption_model import EncoderDecoderModel
from src.data.dataset import ChestXrayDataset
from src.data.vocabulary import Vocabulary
from src.data.transforms import get_transforms
from src.data.collate import collate_fn
from src.training.trainer import CaptionTrainer
from src.training.metrics import CaptionMetrics
from src.utils.device_check import get_device, get_device_info, print_device_info

print("✓ All modules imported successfully!")

✓ All modules imported successfully!


In [3]:
# Matplotlib configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✓ Visualization settings configured")

✓ Visualization settings configured


## 2. Configuration

The project uses YAML configuration files for all hyperparameters:

- **model_config.yaml**: Model architecture, training hyperparameters, inference settings
- **data_config.yaml**: Data processing, splits, vocabulary settings

In [4]:
# Load configurations
config_path = project_root / 'configs' / 'model_config.yaml'
data_config_path = project_root / 'configs' / 'data_config.yaml'

with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

with open(data_config_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("✓ Configuration files loaded")

✓ Configuration files loaded


In [5]:
# Device detection
if torch.cuda.is_available():
    device = 'cuda'
    print(f"Using device: CUDA - {torch.cuda.get_device_name(0)}")
elif torch.backends.mps.is_available():
    device = 'mps'
    print("Using device: MPS (Apple Silicon)")
else:
    device = 'cpu'
    print("Using device: CPU")

# Display key hyperparameters
print("\nTraining Configuration:")
print("="*60)
print(f"  Epochs: {config['training']['num_epochs']}")
print(f"  Batch Size: {config['training']['batch_size']}")
print(f"  Learning Rate: {config['training']['optimizer']['learning_rate']}")
print(f"  Gradient Clip: {config['training']['gradient_clip_norm']}")
print(f"  Early Stopping Patience: {config['training']['early_stopping']['patience']}")
print(f"  Inference Method: {config['inference']['decoding']['method']}")
print("="*60)

Using device: MPS (Apple Silicon)

Training Configuration:
  Epochs: 30
  Batch Size: 32
  Learning Rate: 0.0001
  Gradient Clip: 5.0
  Early Stopping Patience: 10
  Inference Method: beam_search


## 3. Data Loading

The data pipeline follows this flow:

1. **Vocabulary**: Load 514 tokens built from training data only (prevents leakage)
2. **Transforms**: 
   - Train: Random rotation, color jitter (NO horizontal flip to preserve anatomical laterality)
   - Val/Test: Center crop and normalization only
3. **Dataset**: PyTorch Dataset with proper padding and special tokens
4. **DataLoader**: Custom collate function sorts captions by length for efficient packing

**Important**: Patient-level splits ensure no data leakage between train/val/test.

In [6]:
# Define paths
processed_dir = project_root / 'data' / 'processed' / 'first_frontal_impression'
image_dir = project_root / 'data' / 'images' / 'images_normalized'

# Load vocabulary
vocab = Vocabulary(str(processed_dir / 'vocabulary.txt'))
print(f"✓ Vocabulary loaded: {len(vocab)} tokens")
print(f"  Special tokens: PAD={vocab.PAD_IDX}, START={vocab.START_IDX}, END={vocab.END_IDX}, UNK={vocab.UNK_IDX}")

✓ Vocabulary loaded: 514 tokens
  Special tokens: PAD=0, START=1, END=2, UNK=3


In [7]:
# Create transforms
train_transform = get_transforms(mode='train', image_size=224)
val_transform = get_transforms(mode='val', image_size=224)

print("✓ Image transforms created")
print("  Train: RandomRotation + ColorJitter + Normalize")
print("  Val/Test: CenterCrop + Normalize")

✓ Image transforms created
  Train: RandomRotation + ColorJitter + Normalize
  Val/Test: CenterCrop + Normalize


In [8]:
# Create datasets
train_dataset = ChestXrayDataset(
    csv_file=str(processed_dir / 'train.csv'),
    image_dir=str(image_dir),
    vocabulary=vocab,
    transform=train_transform,
    max_caption_length=config['data']['max_caption_length']
)

val_dataset = ChestXrayDataset(
    csv_file=str(processed_dir / 'val.csv'),
    image_dir=str(image_dir),
    vocabulary=vocab,
    transform=val_transform,
    max_caption_length=config['data']['max_caption_length']
)

test_dataset = ChestXrayDataset(
    csv_file=str(processed_dir / 'test.csv'),
    image_dir=str(image_dir),
    vocabulary=vocab,
    transform=val_transform,
    max_caption_length=config['data']['max_caption_length']
)

print(f"\nDataset sizes:")
print(f"  Train: {len(train_dataset)} samples")
print(f"  Val:   {len(val_dataset)} samples")
print(f"  Test:  {len(test_dataset)} samples")


Dataset sizes:
  Train: 2948 samples
  Val:   368 samples
  Test:  370 samples


In [9]:
# Create data loaders
batch_size = config['training']['batch_size']
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=config['data']['num_workers'],
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=config['data']['num_workers'],
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,  # Disable for test (cleaner output)
    pin_memory=True
)

print(f"✓ Data loaders created")
print(f"  Batch size: {batch_size}")
print(f"  Batches per epoch: Train={len(train_loader)}, Val={len(val_loader)}, Test={len(test_loader)}")

✓ Data loaders created
  Batch size: 32
  Batches per epoch: Train=93, Val=12, Test=12


In [10]:
# Create model
model = EncoderDecoderModel(
    vocab_size=len(vocab),
    encoder_architecture=config['model']['encoder']['architecture'],
    encoder_pretrained=config['model']['encoder']['pretrained'],
    encoder_freeze=config['model']['encoder']['freeze_backbone'],
    encoder_feature_dim=config['model']['encoder']['output_feature_dim'],
    attention_type=config['model']['decoder']['attention']['type'],
    attention_dim=config['model']['decoder']['attention']['attention_dim'],
    embedding_dim=config['model']['decoder']['embedding_dim'],
    decoder_dim=config['model']['decoder']['hidden_dim'],
    num_decoder_layers=config['model']['decoder']['num_layers'],
    dropout=config['model']['decoder']['dropout'],
    pad_idx=vocab.PAD_IDX,
    start_idx=vocab.START_IDX,
    end_idx=vocab.END_IDX,
    unk_idx=vocab.UNK_IDX
)

model = model.to(device)

print("✓ Model created successfully")

✓ Model created successfully


In [11]:
# Display model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Model Architecture:")
print("="*60)
print(f"  Total Parameters: {total_params:,}")
print(f"  Trainable Parameters: {trainable_params:,}")
print(f"  Frozen Parameters: {total_params - trainable_params:,}")
print(f"  Memory: ~{total_params * 4 / (1024**2):.1f} MB (FP32)")
print("="*60)

Model Architecture:
  Total Parameters: 22,412,139
  Trainable Parameters: 15,458,283
  Frozen Parameters: 6,953,856
  Memory: ~85.5 MB (FP32)


In [12]:
# Test forward pass
print("\nTesting forward pass...")
sample_batch = next(iter(train_loader))
images, captions, caption_lengths, _, _ = sample_batch
images = images.to(device)
captions = captions.to(device)
caption_lengths = caption_lengths.to(device)

with torch.no_grad():
    predictions, attention_weights, sorted_captions, sorted_lengths = model(
        images, captions[:, :-1], caption_lengths - 1
    )

print(f"✓ Forward pass successful!")
print(f"  Input images: {images.shape}")
print(f"  Predictions: {predictions.shape}")
print(f"  Attention weights shape: {attention_weights.shape}")


Testing forward pass...
✓ Forward pass successful!
  Input images: torch.Size([32, 3, 224, 224])
  Predictions: torch.Size([32, 48, 514])
  Attention weights shape: torch.Size([32, 48, 49])


## 5. Training Execution

The **CaptionTrainer** handles the complete training loop:

### Training Loop (per epoch):
1. **Forward pass**: Images → Encoder → Attention → Decoder → Predictions
2. **Loss calculation**: Cross-entropy with label smoothing + perplexity
3. **Backpropagation**: Gradient clipping to prevent exploding gradients
4. **Validation**: Generate captions with beam search, compute BLEU/METEOR/ROUGE-L
5. **Checkpointing**: Save best model based on validation BLEU-4
6. **Early stopping**: Stop if validation metrics don't improve for N epochs

### Features:
- Mixed precision training (FP16) for faster training on compatible GPUs
- Learning rate scheduling (ReduceLROnPlateau)
- Progress bars with tqdm
- Comprehensive logging (metrics CSV, training manifest JSON)

**Note**: Training can take 2-4 hours on GPU, 8-12 hours on CPU.

In [13]:
# Create output directory with timestamp
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
variant_name = f"{config['model']['encoder']['architecture']}_{config['model']['decoder']['attention']['type']}"
output_dir = project_root / 'outputs' / 'training_runs' / f"{variant_name}_{timestamp}"

print(f"Training output directory: {output_dir}")

Training output directory: /Users/marialinhares/uni/masters/1year1semester/MCIM/MCIM_Final_Project/outputs/training_runs/densenet121_bahdanau_20251230_224113


In [14]:
# Create trainer
trainer = CaptionTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    vocabulary=vocab,
    config=config,
    device=device,
    output_dir=str(output_dir),
    resume_checkpoint=None  # Set path to resume training
)

print("✓ Trainer initialized")

✓ Trainer initialized


In [15]:
# Run training
print("\nStarting training...")
print("="*60)

trainer.train()

print("\n✓ Training complete!")
print(f"✓ Best model saved to: {output_dir / 'checkpoints' / 'best_model.pt'}")


Starting training...
Starting training for 30 epochs
Output directory: /Users/marialinhares/uni/masters/1year1semester/MCIM/MCIM_Final_Project/outputs/training_runs/densenet121_bahdanau_20251230_224113/first_frontal_impression_20251230_224113
Device: mps
Mixed precision: False
Batch size: 32


Epoch 1 [Train]:   0%|          | 0/93 [00:19<?, ?it/s]


ValueError: Expected input batch_size (1056) to match target batch_size (1536).

## 6. Training Visualization

Analyze training dynamics through loss curves and validation metrics:

- **Loss & Perplexity**: Should decrease over epochs (lower is better)
- **BLEU Scores**: Should increase over epochs (higher is better, max 1.0)
- **METEOR & ROUGE-L**: Alternative metrics that consider synonyms and word order

### Expected Behavior:
- Training loss decreases smoothly
- Validation loss follows training loss (no severe overfitting)
- BLEU-4 typically reaches 0.15-0.30 for medical image captioning
- Learning rate decreases when validation metrics plateau

In [ ]:
# Load metrics
metrics_df = pd.read_csv(output_dir / 'metrics.csv')

print(f"Training Metrics Loaded:")
print(f"  Total epochs trained: {len(metrics_df)}")
print(f"\nFirst 5 epochs:")
display(metrics_df.head())

In [ ]:
# Plot loss curves
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Train/Val Loss
axes[0, 0].plot(metrics_df['epoch'], metrics_df['train_loss'], 'o-', label='Train', linewidth=2)
axes[0, 0].plot(metrics_df['epoch'], metrics_df['val_loss'], 's-', label='Validation', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Training & Validation Loss')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Perplexity
axes[0, 1].plot(metrics_df['epoch'], metrics_df['train_perplexity'], 'o-', label='Train', linewidth=2)
axes[0, 1].plot(metrics_df['epoch'], metrics_df['val_perplexity'], 's-', label='Validation', linewidth=2)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Perplexity')
axes[0, 1].set_title('Perplexity (lower is better)')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# BLEU Scores
axes[1, 0].plot(metrics_df['epoch'], metrics_df['val_bleu_4'], 'o-', label='BLEU-4', linewidth=2)
axes[1, 0].plot(metrics_df['epoch'], metrics_df['val_bleu_3'], 's-', label='BLEU-3', linewidth=2)
axes[1, 0].plot(metrics_df['epoch'], metrics_df['val_bleu_2'], '^-', label='BLEU-2', linewidth=2)
axes[1, 0].plot(metrics_df['epoch'], metrics_df['val_bleu_1'], 'v-', label='BLEU-1', linewidth=2)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('BLEU Score')
axes[1, 0].set_title('BLEU Scores on Validation Set')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# METEOR & ROUGE-L
axes[1, 1].plot(metrics_df['epoch'], metrics_df['val_meteor'], 'o-', label='METEOR', linewidth=2)
axes[1, 1].plot(metrics_df['epoch'], metrics_df['val_rouge_l'], 's-', label='ROUGE-L', linewidth=2)
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_title('METEOR & ROUGE-L on Validation Set')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
save_path = output_dir / 'training_curves.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"✓ Saved to: {save_path}")
plt.show()

In [ ]:
# Find best epoch
best_epoch = metrics_df.loc[metrics_df['val_bleu_4'].idxmax()]

print(f"\nBest Epoch: {int(best_epoch['epoch'])}")
print("="*60)
print(f"  Val Loss: {best_epoch['val_loss']:.4f}")
print(f"  Val Perplexity: {best_epoch['val_perplexity']:.4f}")
print(f"  BLEU-4: {best_epoch['val_bleu_4']:.4f}")
print(f"  BLEU-3: {best_epoch['val_bleu_3']:.4f}")
print(f"  BLEU-2: {best_epoch['val_bleu_2']:.4f}")
print(f"  BLEU-1: {best_epoch['val_bleu_1']:.4f}")
print(f"  METEOR: {best_epoch['val_meteor']:.4f}")
print(f"  ROUGE-L: {best_epoch['val_rouge_l']:.4f}")
print("="*60)

## 7. Test Set Evaluation

Evaluate the best model on the held-out test set for final performance assessment.

### Evaluation Process:
1. Load best checkpoint (highest validation BLEU-4)
2. Generate captions for all test images using beam search
3. Compute metrics: BLEU-1/2/3/4, METEOR, ROUGE-L
4. Compare test vs validation performance

### Expected Results:
- Test metrics should be similar to validation metrics
- Large gap indicates overfitting
- BLEU-4 typically ranges from 0.15-0.30 for medical images

In [ ]:
# Load best checkpoint
from src.utils.checkpoint import CheckpointManager

checkpoint_mgr = CheckpointManager(str(output_dir / 'checkpoints'))
checkpoint_info = checkpoint_mgr.load_checkpoint(
    str(output_dir / 'checkpoints' / 'best_model.pt'),
    model=model,
    device=device
)

print(f"✓ Loaded best model from epoch {checkpoint_info['epoch']}")
print(f"  Validation BLEU-4: {checkpoint_info.get('metrics', {}).get('val_bleu_4', 'N/A')}")

In [ ]:
# Evaluate on test set
model.eval()
metrics_calculator = CaptionMetrics()
all_references, all_hypotheses = [], []
test_samples = []

print("\nGenerating captions for test set...")
with torch.no_grad():
    for images, captions, caption_lengths, image_paths, uids in tqdm(test_loader):
        images = images.to(device)

        for i in range(len(images)):
            # Generate caption
            method = config['inference']['decoding']['method']
            beam_size = 1 if method == 'greedy' else config['inference']['decoding']['beam_size']

            generated_ids, attention_weights = model.generate_caption(
                images[i:i+1],
                max_length=config['inference']['decoding']['max_length'],
                beam_size=beam_size,
                length_penalty=config['inference']['decoding'].get('length_penalty', 0.0)
            )

            # Decode reference
            reference_tokens = [
                vocab.idx_to_token.get(idx, '<UNK>')
                for idx in captions[i].cpu().tolist()
                if idx not in [vocab.PAD_IDX, vocab.START_IDX]
            ]
            if vocab.END_TOKEN in reference_tokens:
                reference_tokens = reference_tokens[:reference_tokens.index(vocab.END_TOKEN)]

            # Decode hypothesis
            hypothesis_tokens = [
                vocab.idx_to_token.get(idx, '<UNK>')
                for idx in generated_ids
                if idx not in [vocab.PAD_IDX, vocab.START_IDX]
            ]
            if vocab.END_TOKEN in hypothesis_tokens:
                hypothesis_tokens = hypothesis_tokens[:hypothesis_tokens.index(vocab.END_TOKEN)]

            all_references.append([reference_tokens])
            all_hypotheses.append(hypothesis_tokens)

            # Store samples for visualization
            if len(test_samples) < 20:
                test_samples.append({
                    'uid': int(uids[i]),
                    'image_path': image_paths[i],
                    'reference': ' '.join(reference_tokens),
                    'generated': ' '.join(hypothesis_tokens),
                    'attention_weights': attention_weights
                })

print(f"✓ Generated {len(all_hypotheses)} captions")

In [ ]:
# Calculate test metrics
test_metrics = metrics_calculator.compute_all_metrics(all_references, all_hypotheses)

print("\nTest Set Results:")
print("="*60)
for metric_name, value in test_metrics.items():
    print(f"  {metric_name.upper()}: {value:.4f}")
print("="*60)

In [ ]:
# Compare test vs validation
val_metrics = {
    'bleu_4': best_epoch['val_bleu_4'],
    'bleu_3': best_epoch['val_bleu_3'],
    'bleu_2': best_epoch['val_bleu_2'],
    'bleu_1': best_epoch['val_bleu_1'],
    'meteor': best_epoch['val_meteor'],
    'rouge_l': best_epoch['val_rouge_l']
}

# Plot comparison
fig, ax = plt.subplots(figsize=(12, 6))
metrics_names = list(test_metrics.keys())
test_values = [test_metrics[m] for m in metrics_names]
val_values = [val_metrics[m] for m in metrics_names]

x = np.arange(len(metrics_names))
width = 0.35

ax.bar(x - width/2, val_values, width, label='Validation', alpha=0.8)
ax.bar(x + width/2, test_values, width, label='Test', alpha=0.8)

ax.set_ylabel('Score')
ax.set_title('Test vs Validation Metrics', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([m.upper() for m in metrics_names])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
save_path = output_dir / 'test_vs_val_metrics.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Saved to: {save_path}")
plt.show()

## 8. Sample Predictions

Qualitative assessment of generated captions. Good captions should:
- Mention key anatomical structures (heart, lungs, etc.)
- Identify abnormalities if present (atelectasis, effusion, etc.)
- Use appropriate medical terminology
- Be concise and clear (similar to radiologist impressions)

In [ ]:
# Display sample predictions
print("Sample Predictions:")
print("="*80)

for idx, sample in enumerate(test_samples[:10], 1):
    print(f"\n[Sample {idx}]")
    print(f"  Reference:  {sample['reference']}")
    print(f"  Generated:  {sample['generated']}")
    print("-"*80)

## 9. Attention Visualization

Visualize attention weights to understand where the model "looks" when generating each word.

### Interpretation:
- **Hot colors (red/yellow)**: High attention (model focuses here)
- **Cool colors (blue)**: Low attention
- Attention should focus on relevant anatomical regions
- Example: When generating "heart", attention should focus on cardiac silhouette

This provides interpretability and can help identify model failures.

In [ ]:
def visualize_attention(image_path, caption_words, attention_weights, save_path=None):
    """Visualize attention heatmaps over image for each word."""
    from scipy.ndimage import zoom
    import matplotlib.gridspec as gridspec

    img = Image.open(image_path).convert('RGB')
    img_array = np.array(img)

    num_words = min(len(caption_words), 8)
    fig = plt.figure(figsize=(16, 3 * ((num_words + 1) // 2)))
    gs = gridspec.GridSpec((num_words + 1) // 2, 2, figure=fig)

    for idx in range(num_words):
        ax = fig.add_subplot(gs[idx // 2, idx % 2])

        # Reshape 49-dim attention to 7x7, then resize to image size
        attn = attention_weights[idx].cpu().numpy().reshape(7, 7)
        zoom_factor = img_array.shape[0] / 7
        attn_resized = zoom(attn, zoom_factor, order=1)

        ax.imshow(img_array, alpha=0.7)
        im = ax.imshow(attn_resized, cmap='jet', alpha=0.5)
        ax.set_title(f'"{caption_words[idx]}"', fontsize=14, fontweight='bold')
        ax.axis('off')
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)

    plt.suptitle('Attention: Where Model Looks for Each Word', fontsize=16, fontweight='bold')
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

print("✓ Attention visualization function defined")

In [ ]:
# Visualize 3 samples
for idx in range(min(3, len(test_samples))):
    sample = test_samples[idx]
    caption_words = sample['generated'].split()

    print(f"\nSample {idx+1}: {sample['generated']}")

    save_path = output_dir / f'attention_sample_{idx+1}.png'
    visualize_attention(
        sample['image_path'],
        caption_words,
        sample['attention_weights'],
        save_path=save_path
    )
    print(f"✓ Saved to: {save_path}")

## 10. Error Analysis

Identify failure modes and improvement opportunities by analyzing:
- **Best predictions**: What makes them good? (high BLEU-4)
- **Worst predictions**: Common errors (low BLEU-4)
- **Caption length correlation**: Does model struggle with long/short captions?
- **BLEU distribution**: Are most captions good, or bimodal?

### Common Error Patterns:
1. **Generic captions**: Model produces vague descriptions ("no acute findings")
2. **Missing details**: Misses specific abnormalities
3. **Repetition**: Repeats words or phrases
4. **Hallucination**: Describes findings not present in image

In [ ]:
# Calculate per-sample BLEU-4
from nltk.translate.bleu_score import sentence_bleu

per_sample_bleu = []
for ref, hyp in zip(all_references, all_hypotheses):
    bleu = sentence_bleu(ref, hyp, weights=(0, 0, 0, 1))
    per_sample_bleu.append(bleu)

print(f"Per-sample BLEU-4 statistics:")
print(f"  Mean: {np.mean(per_sample_bleu):.4f}")
print(f"  Median: {np.median(per_sample_bleu):.4f}")
print(f"  Std: {np.std(per_sample_bleu):.4f}")
print(f"  Min: {np.min(per_sample_bleu):.4f}")
print(f"  Max: {np.max(per_sample_bleu):.4f}")

In [ ]:
# Best predictions
best_indices = np.argsort(per_sample_bleu)[-5:][::-1]
print("Best Predictions (Highest BLEU-4):")
print("="*80)
for rank, idx in enumerate(best_indices, 1):
    print(f"\n[Rank {rank}] BLEU-4: {per_sample_bleu[idx]:.4f}")
    print(f"  Reference:  {' '.join(all_references[idx][0])}")
    print(f"  Generated:  {' '.join(all_hypotheses[idx])}")

In [ ]:
# Worst predictions
worst_indices = np.argsort(per_sample_bleu)[:5]
print("\n\nWorst Predictions (Lowest BLEU-4):")
print("="*80)
for rank, idx in enumerate(worst_indices, 1):
    print(f"\n[Rank {rank}] BLEU-4: {per_sample_bleu[idx]:.4f}")
    print(f"  Reference:  {' '.join(all_references[idx][0])}")
    print(f"  Generated:  {' '.join(all_hypotheses[idx])}")

In [ ]:
# BLEU distribution
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(per_sample_bleu, bins=30, alpha=0.7, edgecolor='black')
ax.axvline(np.mean(per_sample_bleu), color='red', linestyle='--', linewidth=2, 
           label=f'Mean: {np.mean(per_sample_bleu):.4f}')
ax.axvline(np.median(per_sample_bleu), color='green', linestyle='--', linewidth=2, 
           label=f'Median: {np.median(per_sample_bleu):.4f}')
ax.set_xlabel('BLEU-4 Score', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Distribution of Per-Sample BLEU-4 Scores on Test Set', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
save_path = output_dir / 'bleu_distribution.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"\n✓ Saved to: {save_path}")
plt.show()

## 11. Summary & Export

Final summary of training results and next steps for improvement.

In [ ]:
# Save test results
test_results = {
    'test_metrics': test_metrics,
    'val_metrics': val_metrics,
    'best_epoch': int(best_epoch['epoch']),
    'total_epochs_trained': len(metrics_df),
    'model_config': config,
    'output_directory': str(output_dir)
}

with open(output_dir / 'test_results.json', 'w') as f:
    json.dump(test_results, f, indent=2)

print("✓ Test results saved")

In [ ]:
# Print final summary
print("\n" + "="*70)
print("TRAINING COMPLETE - SUMMARY")
print("="*70)
print(f"\n📊 Model: {variant_name}")
print(f"📊 Best Epoch: {int(best_epoch['epoch'])}")
print(f"📊 Training Time: {len(metrics_df)} epochs")

print(f"\n📈 Test Set Performance:")
print(f"  • BLEU-4: {test_metrics['bleu_4']:.4f}")
print(f"  • BLEU-3: {test_metrics['bleu_3']:.4f}")
print(f"  • BLEU-2: {test_metrics['bleu_2']:.4f}")
print(f"  • BLEU-1: {test_metrics['bleu_1']:.4f}")
print(f"  • METEOR: {test_metrics['meteor']:.4f}")
print(f"  • ROUGE-L: {test_metrics['rouge_l']:.4f}")

print(f"\n📁 Output Directory: {output_dir}")
print("="*70)

In [ ]:
# List generated files
print("\n📄 Generated Files:")
generated_files = [
    'checkpoints/best_model.pt',
    'metrics.csv',
    'training_manifest.json',
    'test_results.json',
    'training_curves.png',
    'test_vs_val_metrics.png',
    'bleu_distribution.png'
]

for fname in generated_files:
    fpath = output_dir / fname
    if fpath.exists():
        print(f"  ✓ {fname}")
    else:
        print(f"  ✗ {fname} (not found)")

# Count attention visualizations
attention_files = list((output_dir).glob('attention_sample_*.png'))
print(f"  ✓ {len(attention_files)} attention visualization(s)")

## Next Steps

Potential improvements to explore:

1. **Fine-tune Encoder**: Unfreeze DenseNet layers for domain adaptation
   ```python
   model.encoder.unfreeze_layers(from_layer='denseblock3')
   ```

2. **Experiment with Beam Search**: Try different beam sizes (3, 5, 10) and length penalties

3. **Alternative Attention**: Try Luong (multiplicative) attention for comparison

4. **Ensemble Models**: Train multiple models and average predictions

5. **Deploy for Inference**: Create inference script for real-time caption generation

6. **Error Analysis**: Identify systematic failures and retrain on hard examples

---

**Training Complete!** 🎉